In [170]:
# Code to load data from 1st july to fm_status straightaway. Then temp is created to sync 24 hour data(upsert)

In [171]:
#Importing libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import math
from IPython.display import display
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, Float

In [172]:
#Defining function to get a dataframe from sql

def get_df_from_sql(SSH_required, query,key_path):

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [173]:
#Getting the dataframe from sql query on ops main

SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'

query_path = 'pickup_query.sql'
with open(query_path,'r') as file:
    query = file.read()

df = get_df_from_sql(SSH_required, query, key_path)
# df = df[df['warehouse_id'] == 1514]

print(df)

                   key  warehouse_id pickup_vehicle          pickuptime  \
0       100-1514-Delhi          1514      Devraj As 2024-07-01 17:59:23   
1       100-1514-Delhi          1514      Devraj As 2024-07-01 17:59:27   
2       100-1514-Delhi          1514      Devraj As 2024-07-01 17:59:33   
3       100-1514-Delhi          1514      Devraj As 2024-07-01 17:59:36   
4       100-1514-Delhi          1514      Devraj As 2024-07-01 17:59:39   
...                ...           ...            ...                 ...   
372695   87-327-Mumbai           327         vikash 2024-07-12 18:18:48   
372696   87-327-Mumbai           327         vikash 2024-07-12 18:18:55   
372697   87-327-Mumbai           327         vikash 2024-07-13 19:01:38   
372698   87-327-Mumbai           327         vikash 2024-07-13 19:01:42   
372699   87-327-Mumbai           327         vikash 2024-07-13 19:01:49   

                     created_date           awb  
0      2024-07-01 07:15:16.736392  GS1124444273  

In [174]:
#Making pickup slots on warehouses and pickup vehicle

df['pickuptime'] = pd.to_datetime(df['pickuptime'])

results = []

for (key, warehouse, pickup_vehicle), group in df.groupby(['key', 'warehouse_id', 'pickup_vehicle']):
    group = group.reset_index(drop=True)

    pickup_start_time = group.loc[0, 'pickuptime']

    for i in range(1, len(group)):
        current_time = group.loc[i, 'pickuptime']
        previous_time = group.loc[i - 1, 'pickuptime']
        
        # Check the difference between current and previous pickuptime
        if (current_time - previous_time).total_seconds() > 7200:  # More than two hours
            # End the current pickup slot and start a new one
            results.append({
                'key': key,
                'warehouse_id': warehouse,
                'pickup_vehicle': pickup_vehicle,
                'pickup_start_time': pickup_start_time,
                'pickup_end_time': previous_time
            })
            pickup_start_time = current_time  # Update the start time for the new slot
    
    # Append the last slot for the warehouse
    results.append({
        'key': key,
        'warehouse_id': warehouse,
        'pickup_vehicle': pickup_vehicle,
        'pickup_start_time': pickup_start_time,
        'pickup_end_time': group.loc[len(group) - 1, 'pickuptime']
    })

# Create DataFrame from results list
results_df = pd.DataFrame(results)

# Calculate slot_duration in minutes
results_df['slot_duration'] = ((results_df['pickup_end_time'] - results_df['pickup_start_time']).dt.total_seconds() / 60).apply(math.ceil)

# Sort DataFrame by warehouse_id and pickup_start_time (if not already sorted)
results_df.sort_values(by=['key', 'warehouse_id', 'pickup_start_time'], inplace=True)

# Calculate gaps between pickup slots in minutes
# results_df['gap_to_next'] = results_df.groupby(['key', 'warehouse_id', 'pickup_vehicle'])['pickup_start_time'].shift(-1) - results_df['pickup_end_time']
# results_df['gap_to_next'] = (results_df['gap_to_next'].dt.total_seconds() / 60)
# results_df['gap_to_next'].fillna(0, inplace=True)
# results_df['gap_to_next'] = (results_df['gap_to_next'].astype(int) + 1).replace(1, np.nan)

results_df.sort_values(by=['key', 'warehouse_id', 'pickup_vehicle'], inplace=True)

# print(results_df)

In [175]:
# Adding actual load on slots

loads = []

for index, row in results_df.iterrows():
    key = row['key']
    warehouse_id = row['warehouse_id']
    pickup_vehicle = row['pickup_vehicle']
    slot_start_time = row['pickup_start_time']
    slot_end_time = row['pickup_end_time']
    
    load = 0  # Initialize load count for this slot

    filtered_df = df[(df['key'] == key) &
                     (df['warehouse_id'] == warehouse_id) &
                     (df['pickup_vehicle'] == pickup_vehicle) &
                     (df['pickuptime'] >= slot_start_time) &
                     (df['pickuptime'] <= slot_end_time)]

    # Count the number of AWBs within the slot time range and add to load
    load += len(filtered_df)
    
    # Append the total load count for this slot to the loads list
    loads.append(load)

# Assign the 'actual_load' column to results_df using the loads list
results_df['actual_load'] = loads
pickup_slots_df = results_df

# # Print or display the updated DataFrame with the 'actual_load' column
# print(pickup_slots_df)

# # Save DataFrame to Excel
# excel_file = 'pickup_slots.xlsx'
# pickup_slots_df.to_excel(excel_file, index=False)

# print(f"DataFrame saved to {excel_file}")


In [ ]:
#Loading pickup cutoff time data

query_path = 'pickup_cutoff_time.sql'
with open(query_path,'r') as file:
    query = file.read()

pickup_cutoff_df = get_df_from_sql(SSH_required, query, key_path)

pickup_cutoff_df['pickup_cutoff'] = pd.to_datetime(pickup_cutoff_df['pickup_cutoff'], format='%H:%M:%S', errors='coerce').dt.time
# pickup_cutoff_df = pickup_cutoff_df[pickup_cutoff_df['warehouse_id'] == 1514]
display(pickup_cutoff_df)


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,pickup_cutoff,accounts_poc_number,fm_poc_number,client_poc_number,ach_number
0,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,18:30:00,,9066859089,8548848808,
1,100-1514-Delhi,1514,GIVA DEL,GIVA,100,Delhi,17:30:00,,8287823040,7503668054,
2,100-1514-Delhi,1514,GIVA DEL,GIVA,100,Delhi,11:00:00,,8287823040,7503668054,
3,100-329-Bangalore,329,Indiejewel Fashions Private Limited,GIVA,100,Bangalore,18:30:00,,9066859089,8548848808,
4,101-1400-Hyderabad,1400,BW HYD New,Bebodywise,101,Hyderabad,17:00:00,,8008886275,8422966291,
...,...,...,...,...,...,...,...,...,...,...,...
227,81-250-Hyderabad,250,GS_HYD_KMP,HealthKart,81,Hyderabad,19:00:00,,8008886275,9550899104,
228,81-370-Bangalore,370,DS_blr_mtl_HK,HealthKart,81,Bangalore,15:00:00,,9066859089,,
229,81-833-Delhi,833,DS_DEL_HK,HealthKart,81,Delhi,15:00:00,,8287823040,8882625265,
230,87-1025-Delhi,1025,TTL_GGN,The Label Life,87,Delhi,10:30:00,,8287823040,7838467876,


In [ ]:
print(pickup_cutoff_df[(pickup_cutoff_df['key'] == '332-1540-Hyderabad') & (pd.to_datetime(pickup_cutoff_df['pickup_cutoff'], format='%H:%M:%S').dt.time == pd.to_datetime('20:00:00', format='%H:%M:%S').time())])

                   key  warehouse_id warehouse_name  user_name  user_id  \
55  332-1540-Hyderabad          1540  Decathlon HYD  Decathlon      332   

   warehouse_city pickup_cutoff accounts_poc_number fm_poc_number  \
55      Hyderabad      20:00:00                        8008886275   

   client_poc_number ach_number  
55        7373321003             


In [ ]:
#Expanding cut off time data to a cut off datetime data with range of [d-3, d] days i.e. 4 days

# print(datetime.now())

# Step 1: Define the range of dates
start_date = datetime(2024, 6, 29)  # Adjusted start date, datetime.now() is in IST already
end_date = datetime.now()

# Step 2: Create a new DataFrame to hold the expanded rows
expanded_rows = []

# Iterate through each row in pickup_cutoff_df
for index, row in pickup_cutoff_df.iterrows():
    key = row['key']
    warehouse_id = row['warehouse_id']
    warehouse_name = row['warehouse_name']
    user_name = row['user_name']
    user_id = row['user_id']
    warehouse_city = row['warehouse_city']
    pickup_cutoff = row['pickup_cutoff']

    # Iterate through each date in the range
    date_iterator = start_date
    while date_iterator <= end_date:
        # Combine current date with pickup_cutoff_time to create datetime
        pickup_datetime = datetime.combine(date_iterator.date(), pickup_cutoff)

        # Append row with key, pickup_cutoff_time, and current date
        expanded_rows.append({
            'key': key,
            'warehouse_id': warehouse_id,
            'warehouse_name' : warehouse_name,
            'user_name' : user_name,
            'user_id' : user_id,
            'warehouse_city' : warehouse_city,
            'pickup_cutoff': pickup_cutoff,
            'pickup_cutoff_time1': pickup_datetime
        })

        # Move to the next date
        date_iterator += timedelta(days=1)

# Create DataFrame from expanded rows
expanded_df = pd.DataFrame(expanded_rows)

merged_df = pd.merge(pickup_cutoff_df, expanded_df, left_on=['key', 'warehouse_id', 'warehouse_name', 'user_name', 'user_id', 'warehouse_city', 'pickup_cutoff'], right_on=['key', 'warehouse_id', 'warehouse_name', 'user_name', 'user_id', 'warehouse_city', 'pickup_cutoff'], how='left')

# Drop pickup_cutoff_time and rename pickup_cutoff_time1
merged_df.drop(columns=['pickup_cutoff'], inplace=True)
merged_df.rename(columns={'pickup_cutoff_time1': 'pickup_cutoff'}, inplace=True)
pickup_cutoff_df = merged_df

# Display the expanded DataFrame
print(pickup_cutoff_df)

                     key  warehouse_id warehouse_name       user_name  \
0     100-1514-Bangalore          1514       GIVA DEL            GIVA   
1     100-1514-Bangalore          1514       GIVA DEL            GIVA   
2     100-1514-Bangalore          1514       GIVA DEL            GIVA   
3     100-1514-Bangalore          1514       GIVA DEL            GIVA   
4     100-1514-Bangalore          1514       GIVA DEL            GIVA   
...                  ...           ...            ...             ...   
6027       87-327-Mumbai           327           GOAT  The Label Life   
6028       87-327-Mumbai           327           GOAT  The Label Life   
6029       87-327-Mumbai           327           GOAT  The Label Life   
6030       87-327-Mumbai           327           GOAT  The Label Life   
6031       87-327-Mumbai           327           GOAT  The Label Life   

      user_id warehouse_city accounts_poc_number fm_poc_number  \
0         100      Bangalore                        90668

In [ ]:
display(expanded_df)

,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,pickup_cutoff,pickup_cutoff_time1
0,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,18:30:00,2024-06-29 18:30:00
1,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,18:30:00,2024-06-30 18:30:00
2,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,18:30:00,2024-07-01 18:30:00
3,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,18:30:00,2024-07-02 18:30:00
4,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,18:30:00,2024-07-03 18:30:00
...,...,...,...,...,...,...,...,...
6027,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,16:00:00,2024-07-20 16:00:00
6028,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,16:00:00,2024-07-21 16:00:00
6029,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,16:00:00,2024-07-22 16:00:00
6030,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,16:00:00,2024-07-23 16:00:00


In [ ]:
target_datetime = pd.to_datetime('2024-07-18 20:00:00')
print("Target datetime:", target_datetime)
result = pickup_cutoff_df[(pickup_cutoff_df['key'] == '332-1540-Hyderabad') & (pickup_cutoff_df['pickup_cutoff'] == '2024-07-18 20:00:00')]
sorted_result = result.sort_values(by='pickup_cutoff', ascending=False)
# result = result[result['pickup_cutoff'] == '2024-07-18 20:00:00']
display(result)

Target datetime: 2024-07-18 20:00:00


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff
1449,332-1540-Hyderabad,1540,Decathlon HYD,Decathlon,332,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00


In [ ]:
#Matching pickups that happened with every possible pickup cutoff time

final_df = pd.merge(pickup_slots_df, pickup_cutoff_df, left_on = ['key', 'warehouse_id'], right_on = ['key', 'warehouse_id'], how='left')
display(final_df)

,key,warehouse_id,pickup_vehicle,pickup_start_time,pickup_end_time,slot_duration,actual_load,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff
0,100-1514-Delhi,1514,Devraj As,2024-07-01 17:59:23,2024-07-01 18:44:24,46,44,GIVA DEL,GIVA,100.0,Delhi,,8287823040,7503668054,,2024-06-29 17:30:00
1,100-1514-Delhi,1514,Devraj As,2024-07-01 17:59:23,2024-07-01 18:44:24,46,44,GIVA DEL,GIVA,100.0,Delhi,,8287823040,7503668054,,2024-06-30 17:30:00
2,100-1514-Delhi,1514,Devraj As,2024-07-01 17:59:23,2024-07-01 18:44:24,46,44,GIVA DEL,GIVA,100.0,Delhi,,8287823040,7503668054,,2024-07-01 17:30:00
3,100-1514-Delhi,1514,Devraj As,2024-07-01 17:59:23,2024-07-01 18:44:24,46,44,GIVA DEL,GIVA,100.0,Delhi,,8287823040,7503668054,,2024-07-02 17:30:00
4,100-1514-Delhi,1514,Devraj As,2024-07-01 17:59:23,2024-07-01 18:44:24,46,44,GIVA DEL,GIVA,100.0,Delhi,,8287823040,7503668054,,2024-07-03 17:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100670,87-327-Mumbai,327,vikash,2024-07-13 19:01:38,2024-07-13 19:01:49,1,3,GOAT,The Label Life,87.0,Mumbai,,,9996237317,,2024-07-20 16:00:00
100671,87-327-Mumbai,327,vikash,2024-07-13 19:01:38,2024-07-13 19:01:49,1,3,GOAT,The Label Life,87.0,Mumbai,,,9996237317,,2024-07-21 16:00:00
100672,87-327-Mumbai,327,vikash,2024-07-13 19:01:38,2024-07-13 19:01:49,1,3,GOAT,The Label Life,87.0,Mumbai,,,9996237317,,2024-07-22 16:00:00
100673,87-327-Mumbai,327,vikash,2024-07-13 19:01:38,2024-07-13 19:01:49,1,3,GOAT,The Label Life,87.0,Mumbai,,,9996237317,,2024-07-23 16:00:00


In [ ]:
result = final_df[(final_df['key'] == '332-1540-Hyderabad') & (final_df['pickup_cutoff'] == '2024-07-18 20:00:00')]
display(result)

,key,warehouse_id,pickup_vehicle,pickup_start_time,pickup_end_time,slot_duration,actual_load,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff
37798,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-01 21:44:32,2024-07-01 22:28:55,45,224,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
37824,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-02 20:59:52,2024-07-02 21:27:08,28,135,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
37850,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-03 19:39:40,2024-07-03 20:04:27,25,116,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
37876,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-04 20:10:17,2024-07-04 20:32:27,23,115,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
37902,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-05 20:38:55,2024-07-05 21:06:25,28,85,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
37928,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-06 20:55:08,2024-07-06 21:50:11,56,140,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
37954,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-07 20:42:51,2024-07-07 21:22:26,40,145,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
37980,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-08 20:18:25,2024-07-08 21:08:49,51,177,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
38006,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-09 20:02:41,2024-07-09 20:27:42,26,139,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00
38032,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-10 20:26:11,2024-07-10 20:42:19,17,121,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00


In [ ]:
# # There are some warehouse ids that are active but are not in pickup_cutoff_time, we are dropping them, but they need to be added in excel.pickup_cutoff_time

# # Step 1: Identify and store the rows that have NaN in the 'pickup_cutoff' column
# dropped_rows = final_df[final_df['pickup_cutoff'].isna()]

# # Step 2: Drop the rows with NaN in the 'pickup_cutoff' column from final_df
# final_df.dropna(subset=['pickup_cutoff'], inplace=True)

# # Step 3: Select the relevant columns from the dropped rows
# dropped_rows_selected = dropped_rows[['warehouse_city', 'user_name', 'user_id', 'warehouse_name', 'warehouse_id', 'key']]

# # Step 4: Remove duplicate rows
# dropped_rows_selected = dropped_rows_selected.drop_duplicates()

# # Step 5: Save the selected columns from the dropped rows into an Excel file
# output_file_path = 'dropped_warehouses.xlsx'
# dropped_rows_selected.to_excel(output_file_path, index=False)

# print(f"Selected columns from dropped rows have been saved to {output_file_path}")

In [ ]:
# Choosing rows with right pickup cutoff matching

final_df['slot_id'] = final_df['warehouse_id'].astype(str) + '-' + final_df['pickup_vehicle'].astype(str) + '-' + final_df['pickup_start_time'].astype(str)

# Calculate the duration and take the absolute value to see nearest cutoff time
final_df['start_to_pickup_cutoff'] = (final_df['pickup_start_time'] - final_df['pickup_cutoff']).abs()
final_df['end_to_pickup_cutoff'] = (final_df['pickup_end_time'] - final_df['pickup_cutoff']).abs()
# display(final_df)

# Function to apply the rules
def filter_rows(group):
    # Check if pickup_cutoff is null
    if group['pickup_cutoff'].isnull().all():
        # Handle the case when pickup_cutoff is missing
        return group.iloc[[0]]  # Returning the first row, but you can define your custom handling

    # Rule 1: If start_to_pickup_cutoff < 1 hour
    rule_1 = group[group['start_to_pickup_cutoff'] < pd.Timedelta(hours=1)]
    if not rule_1.empty:
        return rule_1.iloc[[0]]

    # Rule 2: If end_to_pickup_cutoff < 1 hour
    rule_2 = group[group['end_to_pickup_cutoff'] < pd.Timedelta(hours=1)]
    if not rule_2.empty:
        return rule_2.iloc[[0]]

    # Rule 3: If pickup_cutoff_time is within pickup_start_time and pickup_end_time
    rule_3 = group[(group['pickup_cutoff'] >= group['pickup_start_time']) & (group['pickup_cutoff'] <= group['pickup_end_time'])]
    if not rule_3.empty:
        return rule_3.iloc[[0]]

    # Rule 4: Closest pickup_cutoff_time < pickup_start_time
    group['diff'] = (group['pickup_start_time'] - group['pickup_cutoff']).abs()
    rule_4 = group[group['pickup_cutoff'] < group['pickup_start_time']]
    if not rule_4.empty:
        return rule_4.loc[[rule_4['diff'].idxmin()]]
    
    return pd.DataFrame()  # If no rule matches, return an empty DataFrame

# Apply the function to each group of slot_id
final_df = final_df.groupby('slot_id').apply(filter_rows).reset_index(drop=True)

# Drop the auxiliary column used for Rule 4
final_df.drop(columns=['diff'], inplace=True)

display(final_df)



,key,warehouse_id,pickup_vehicle,pickup_start_time,pickup_end_time,slot_duration,actual_load,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,slot_id,start_to_pickup_cutoff,end_to_pickup_cutoff
0,87-1025-Delhi,1025,Aakash,2024-07-02 09:49:25,2024-07-02 09:49:43,1,4,TTL_GGN,The Label Life,87.0,Delhi,,8287823040,7838467876,,2024-07-02 10:30:00,1025-Aakash-2024-07-02 09:49:25,0 days 00:40:35,0 days 00:40:17
1,87-1025-Delhi,1025,Aakash,2024-07-03 10:38:19,2024-07-03 10:38:29,1,3,TTL_GGN,The Label Life,87.0,Delhi,,8287823040,7838467876,,2024-07-03 10:30:00,1025-Aakash-2024-07-03 10:38:19,0 days 00:08:19,0 days 00:08:29
2,87-1025-Delhi,1025,Aakash,2024-07-11 09:29:10,2024-07-11 09:29:17,1,2,TTL_GGN,The Label Life,87.0,Delhi,,8287823040,7838467876,,2024-07-10 10:30:00,1025-Aakash-2024-07-11 09:29:10,0 days 22:59:10,0 days 22:59:17
3,87-1025-Delhi,1025,Aakash,2024-07-12 09:43:06,2024-07-12 09:43:09,1,2,TTL_GGN,The Label Life,87.0,Delhi,,8287823040,7838467876,,2024-07-12 10:30:00,1025-Aakash-2024-07-12 09:43:06,0 days 00:46:54,0 days 00:46:51
4,87-1025-Delhi,1025,Aakash,2024-07-13 09:41:08,2024-07-13 09:41:08,0,1,TTL_GGN,The Label Life,87.0,Delhi,,8287823040,7838467876,,2024-07-13 10:30:00,1025-Aakash-2024-07-13 09:41:08,0 days 00:48:52,0 days 00:48:52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,148-953-Delhi,953,Shani,2024-07-18 17:14:47,2024-07-18 17:29:43,15,168,Emiza-Haryana(Tauru),Cureskin,148.0,Delhi,,8287823040,9999307201,,2024-07-18 17:30:00,953-Shani-2024-07-18 17:14:47,0 days 00:15:13,0 days 00:00:17
4140,148-953-Delhi,953,Shani,2024-07-19 16:50:38,2024-07-19 17:49:22,59,160,Emiza-Haryana(Tauru),Cureskin,148.0,Delhi,,8287823040,9999307201,,2024-07-19 17:30:00,953-Shani-2024-07-19 16:50:38,0 days 00:39:22,0 days 00:19:22
4141,148-953-Delhi,953,Shani,2024-07-20 17:12:36,2024-07-20 18:06:43,55,193,Emiza-Haryana(Tauru),Cureskin,148.0,Delhi,,8287823040,9999307201,,2024-07-20 17:30:00,953-Shani-2024-07-20 17:12:36,0 days 00:17:24,0 days 00:36:43
4142,148-953-Delhi,953,Shani,2024-07-21 16:09:50,2024-07-21 17:20:38,71,143,Emiza-Haryana(Tauru),Cureskin,148.0,Delhi,,8287823040,9999307201,,2024-07-21 17:30:00,953-Shani-2024-07-21 16:09:50,0 days 01:20:10,0 days 00:09:22


In [ ]:
result = final_df[(final_df['key'] == '332-1540-Hyderabad') & (final_df['pickup_cutoff'] == '2024-07-18 20:00:00')]
display(result)

,key,warehouse_id,pickup_vehicle,pickup_start_time,pickup_end_time,slot_duration,actual_load,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,slot_id,start_to_pickup_cutoff,end_to_pickup_cutoff
527,332-1540-Hyderabad,1540,Nuruz Jamal choudhury,2024-07-18 19:56:54,2024-07-18 20:10:17,14,90,Decathlon HYD,Decathlon,332.0,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00,1540-Nuruz Jamal choudhury-2024-07-18 19:56:54,0 days 00:03:06,0 days 00:10:17


In [ ]:
# Cleaning data

final_df = final_df[['key', 'user_name', 'warehouse_id', 'pickup_vehicle', 'pickup_cutoff', 'pickup_start_time', 'pickup_end_time', 'actual_load']]
final_df = final_df.sort_values(by= ['key', 'pickup_cutoff'])
final_df = final_df.reset_index(drop=True)

summary_df = final_df.groupby(['key', 'warehouse_id', 'pickup_cutoff']).agg({
    'pickup_start_time': 'min',
    'pickup_end_time': 'max',
    'actual_load': 'sum'
}).reset_index()

# Display the summary DataFrame
display(summary_df)




,key,warehouse_id,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load
0,100-1514-Delhi,1514,2024-07-01 11:00:00,2024-07-01 09:55:56,2024-07-01 10:32:00,57
1,100-1514-Delhi,1514,2024-07-01 17:30:00,2024-07-01 17:59:23,2024-07-01 18:44:24,44
2,100-1514-Delhi,1514,2024-07-02 11:00:00,2024-07-02 09:35:30,2024-07-02 11:02:46,36
3,100-1514-Delhi,1514,2024-07-02 17:30:00,2024-07-02 17:47:27,2024-07-02 18:04:37,45
4,100-1514-Delhi,1514,2024-07-03 11:00:00,2024-07-03 10:50:41,2024-07-03 11:10:30,62
...,...,...,...,...,...,...
2796,87-327-Mumbai,327,2024-07-19 16:00:00,2024-07-19 16:59:29,2024-07-19 16:59:38,3
2797,87-327-Mumbai,327,2024-07-20 16:00:00,2024-07-20 18:16:35,2024-07-20 18:16:44,5
2798,87-327-Mumbai,327,2024-07-21 16:00:00,2024-07-21 17:28:57,2024-07-21 17:29:18,9
2799,87-327-Mumbai,327,2024-07-22 16:00:00,2024-07-22 18:18:23,2024-07-22 18:18:46,8


In [ ]:
# # Filter rows where pickup_cutoff is null
# summary_df = final_df[final_df['pickup_cutoff'].isnull()]

# # Display the filtered DataFrame
# display(summary_df)

In [ ]:
# Ensures Missed Pickups are there

# expanded_df.drop(columns=['pickup_cutoff'], inplace=True)
# expanded_df.rename(columns={'pickup_cutoff_time1':'pickup_cutoff'}, inplace=True)
# expanded_df.sort_values(by=['key', 'pickup_cutoff'], inplace=True)
merged_df.sort_values(by=['key', 'pickup_cutoff'], inplace=True)


print(merged_df)


                     key  warehouse_id warehouse_name       user_name  \
0     100-1514-Bangalore          1514       GIVA DEL            GIVA   
1     100-1514-Bangalore          1514       GIVA DEL            GIVA   
2     100-1514-Bangalore          1514       GIVA DEL            GIVA   
3     100-1514-Bangalore          1514       GIVA DEL            GIVA   
4     100-1514-Bangalore          1514       GIVA DEL            GIVA   
...                  ...           ...            ...             ...   
6027       87-327-Mumbai           327           GOAT  The Label Life   
6028       87-327-Mumbai           327           GOAT  The Label Life   
6029       87-327-Mumbai           327           GOAT  The Label Life   
6030       87-327-Mumbai           327           GOAT  The Label Life   
6031       87-327-Mumbai           327           GOAT  The Label Life   

      user_id warehouse_city accounts_poc_number fm_poc_number  \
0         100      Bangalore                        90668

In [ ]:
result = merged_df[(merged_df['key'] == '332-1540-Hyderabad') & (merged_df['pickup_cutoff'] == '2024-07-18 20:00:00')]
display(result)

,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff
1449,332-1540-Hyderabad,1540,Decathlon HYD,Decathlon,332,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00


In [ ]:
# Creating final table

final = pd.merge(merged_df, summary_df, left_on=['key', 'warehouse_id', 'pickup_cutoff'], right_on=['key', 'warehouse_id', 'pickup_cutoff'], how='left')
display(final)

print(final[(final['key'] == '100-329-Bangalore') & (final['pickup_cutoff'] == '2024-07-17 18:30:00')])


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load
0,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-06-29 18:30:00,NaT,NaT,NaN
1,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-06-30 18:30:00,NaT,NaT,NaN
2,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-01 18:30:00,NaT,NaT,NaN
3,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-02 18:30:00,NaT,NaT,NaN
4,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-03 18:30:00,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6027,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-20 16:00:00,2024-07-20 18:16:35,2024-07-20 18:16:44,5.0
6028,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-21 16:00:00,2024-07-21 17:28:57,2024-07-21 17:29:18,9.0
6029,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-22 16:00:00,2024-07-22 18:18:23,2024-07-22 18:18:46,8.0
6030,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-23 16:00:00,2024-07-23 17:59:27,2024-07-23 17:59:33,4.0


                  key  warehouse_id                       warehouse_name  \
96  100-329-Bangalore           329  Indiejewel Fashions Private Limited   

   user_name  user_id warehouse_city accounts_poc_number fm_poc_number  \
96      GIVA      100      Bangalore                        9066859089   

   client_poc_number ach_number       pickup_cutoff   pickup_start_time  \
96        8548848808            2024-07-17 18:30:00 2024-07-17 21:19:14   

       pickup_end_time  actual_load  
96 2024-07-18 10:58:59        170.0  


In [ ]:
result = final[(final['key'] == '332-1540-Hyderabad') & (final['pickup_cutoff'] == '2024-07-18 20:00:00')]
display(result)

,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load
1501,332-1540-Hyderabad,1540,Decathlon HYD,Decathlon,332,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00,2024-07-18 19:56:54,2024-07-18 20:10:17,90.0


In [ ]:
# Creating expected load

# Ensure pickup_cutoff is a datetime column
final['pickup_cutoff'] = pd.to_datetime(final['pickup_cutoff'])
df['created_date'] = pd.to_datetime(df['created_date'])

result = final[(final['key'] == '332-1540-Hyderabad') & (final['pickup_cutoff'] == '2024-07-18 20:00:00')]
display(result)

# Sort final DataFrame
final_sorted = final.sort_values(by=['key', 'pickup_cutoff'], ascending=[True, False])
display(final_sorted)
result = final_sorted[(final_sorted['key'] == '332-1540-Hyderabad') & (final_sorted['pickup_cutoff'] == '2024-07-18 20:00:00')]
display(result)


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load
1501,332-1540-Hyderabad,1540,Decathlon HYD,Decathlon,332,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00,2024-07-18 19:56:54,2024-07-18 20:10:17,90.0


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load
25,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-24 18:30:00,NaT,NaT,NaN
24,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-23 18:30:00,NaT,NaT,NaN
23,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-22 18:30:00,NaT,NaT,NaN
22,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-21 18:30:00,NaT,NaT,NaN
21,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-20 18:30:00,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-03 16:00:00,2024-07-03 17:27:46,2024-07-03 17:27:46,1.0
6009,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-02 16:00:00,2024-07-02 18:00:52,2024-07-02 18:00:52,1.0
6008,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-01 16:00:00,2024-07-01 17:19:16,2024-07-01 17:19:34,7.0
6007,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-06-30 16:00:00,NaT,NaT,NaN


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load
1501,332-1540-Hyderabad,1540,Decathlon HYD,Decathlon,332,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00,2024-07-18 19:56:54,2024-07-18 20:10:17,90.0


In [ ]:

# Get the previous pickup_cutoff time for each key
final_sorted['previous_cutoff'] = final_sorted.groupby('key')['pickup_cutoff'].shift(-1)

# Define a function to count rows in df that fall between pickup_cutoff and previous_cutoff
def count_rows_between_cutoffs(row, df):
    if pd.isna(row['previous_cutoff']):
        return 0
    return df[(df['key'] == row['key']) & 
              (df['created_date'] >= row['previous_cutoff']) & 
              (df['created_date'] < row['pickup_cutoff'])].shape[0]

# Apply the function to each row in final_sorted
final_sorted['expected_load'] = final_sorted.apply(count_rows_between_cutoffs, axis=1, df=df)

display(final_sorted)
result = final_sorted[(final_sorted['key'] == '332-1540-Hyderabad') & (final_sorted['pickup_cutoff'] == '2024-07-18 20:00:00')]
display(result)


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,previous_cutoff,expected_load
25,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-24 18:30:00,NaT,NaT,NaN,2024-07-23 18:30:00,0
24,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-23 18:30:00,NaT,NaT,NaN,2024-07-22 18:30:00,0
23,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-22 18:30:00,NaT,NaT,NaN,2024-07-21 18:30:00,0
22,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-21 18:30:00,NaT,NaT,NaN,2024-07-20 18:30:00,0
21,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-20 18:30:00,NaT,NaT,NaN,2024-07-19 18:30:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-03 16:00:00,2024-07-03 17:27:46,2024-07-03 17:27:46,1.0,2024-07-02 16:00:00,0
6009,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-02 16:00:00,2024-07-02 18:00:52,2024-07-02 18:00:52,1.0,2024-07-01 16:00:00,2
6008,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-01 16:00:00,2024-07-01 17:19:16,2024-07-01 17:19:34,7.0,2024-06-30 16:00:00,7
6007,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-06-30 16:00:00,NaT,NaT,NaN,2024-06-29 16:00:00,0


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,previous_cutoff,expected_load
1501,332-1540-Hyderabad,1540,Decathlon HYD,Decathlon,332,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00,2024-07-18 19:56:54,2024-07-18 20:10:17,90.0,2024-07-17 20:00:00,92


In [ ]:

#Drop the previous_cutoff column
final_sorted.drop(columns=['previous_cutoff'], inplace=True)
final_filtered = final_sorted

# Assign final_sorted back to final
final = final_filtered



# Display the resulting DataFrame
# display(final)

In [ ]:
display(final)

,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,expected_load
25,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-24 18:30:00,NaT,NaT,NaN,0
24,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-23 18:30:00,NaT,NaT,NaN,0
23,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-22 18:30:00,NaT,NaT,NaN,0
22,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-21 18:30:00,NaT,NaT,NaN,0
21,100-1514-Bangalore,1514,GIVA DEL,GIVA,100,Bangalore,,9066859089,8548848808,,2024-07-20 18:30:00,NaT,NaT,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-03 16:00:00,2024-07-03 17:27:46,2024-07-03 17:27:46,1.0,0
6009,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-02 16:00:00,2024-07-02 18:00:52,2024-07-02 18:00:52,1.0,2
6008,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-01 16:00:00,2024-07-01 17:19:16,2024-07-01 17:19:34,7.0,7
6007,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-06-30 16:00:00,NaT,NaT,NaN,0


In [ ]:
result = final[(final['key'] == '100-1514-Delhi') & (final['pickup_cutoff'] == '2024-07-18 11:00:00')]
display(result)

,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,expected_load
64,100-1514-Delhi,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-18 11:00:00,2024-07-18 10:55:39,2024-07-18 10:59:25,11.0,44


In [ ]:
result = final[(final['key'] == '332-1540-Hyderabad') & (final['pickup_cutoff'] == '2024-07-18 20:00:00')]
display(result)

,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,expected_load
1501,332-1540-Hyderabad,1540,Decathlon HYD,Decathlon,332,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00,2024-07-18 19:56:54,2024-07-18 20:10:17,90.0,92


In [ ]:
print(final[(final['key'] == '332-1540-Hyderabad') & (final['pickup_cutoff'] == '2024-07-18 20:00:00')])

# Removing warehouses with no activity in all days

# Find all key values where pickup_start_time is NaT on all dates and remove them
key_with_all_NaT = final.groupby('key').filter(lambda x: x['pickup_start_time'].isna().all())['key'].unique()

# Convert the result to a list
key_with_all_NaT_list = key_with_all_NaT.tolist()

# Print the list
print(key_with_all_NaT_list)

final = final[~final['key'].isin(key_with_all_NaT_list)]
# display(final)

                     key  warehouse_id warehouse_name  user_name  user_id  \
1501  332-1540-Hyderabad          1540  Decathlon HYD  Decathlon      332   

     warehouse_city accounts_poc_number fm_poc_number client_poc_number  \
1501      Hyderabad                        8008886275        7373321003   

     ach_number       pickup_cutoff   pickup_start_time     pickup_end_time  \
1501            2024-07-18 20:00:00 2024-07-18 19:56:54 2024-07-18 20:10:17   

      actual_load  expected_load  
1501         90.0             92  
['100-1514-Bangalore', '185-359-Hyderabad', '222-1394-Mumbai', '31-1534-Delhi', '31-1585-Bangalore', '31-1892-Mumbai', '31-65-Mumbai', '327-182-Bangalore', '334-1491-Bangalore', '340-1449-Delhi', '341-1453-Delhi', '359-1373-Delhi', '364-1558-Mumbai', '364-1590-Bangalore', '364-1590-Hyderabad', '364-1631-Hyderabad', '364-1721-Delhi', '364-1721-Jaipur', '364-1721-NCR', '374-1554-Delhi', '376-1661-Bangalore', '376-1670-Bangalore', '376-1672-Mumbai', '376-1710-Bang

In [ ]:
current_time = datetime.now()

In [ ]:
print(current_time)

2024-07-24 13:39:38.357954


In [ ]:
# Creating status column

# Step 1: Check if 'pickup_status' column exists, if not, initialize it
if 'pickup_status' not in final.columns:
    final['pickup_status'] = np.nan

# Step 2: Preserve the 'Missed' status
missed_mask = final['pickup_status'] == 'Missed'

# Step 3: Define the conditions
conditions = [
    (final['pickup_start_time'].notna() & (final['pickup_start_time'] <= final['pickup_cutoff'] + pd.Timedelta(hours=1))),
    (final['pickup_start_time'].notna() & (final['pickup_start_time'] > final['pickup_cutoff'] + pd.Timedelta(hours=1))),
    (final['pickup_start_time'].isna() & (final['pickup_cutoff'] + pd.Timedelta(hours=6, minutes=30) < current_time)),
    (final['pickup_start_time'].isna() & (final['pickup_cutoff'] + pd.Timedelta(minutes=30) <= current_time)),
    (final['pickup_start_time'].isna() & (final['pickup_cutoff'] + pd.Timedelta(minutes=30) > current_time))
]

# Define the corresponding values for the conditions
choices = [
    'On-time pickup',
    'Late pickup',
    'Missed',
    'Delayed',
    'Pending'
]

# Step 4: Apply the conditions to update the 'pickup_status' column
final['pickup_status'] = np.select(conditions, choices, default='Unknown')

# Step 5: Reapply the 'Missed' status
final.loc[missed_mask, 'pickup_status'] = 'Missed'
display(final[(final['key'] == '332-1540-Hyderabad') & (final['pickup_cutoff'] == '2024-07-18 20:00:00')])
# Display the final DataFrame
display(final)


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,expected_load,pickup_status
1501,332-1540-Hyderabad,1540,Decathlon HYD,Decathlon,332,Hyderabad,,8008886275,7373321003,,2024-07-18 20:00:00,2024-07-18 19:56:54,2024-07-18 20:10:17,90.0,92,On-time pickup


,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,expected_load,pickup_status
77,100-1514-Delhi,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-24 17:30:00,NaT,NaT,NaN,0,Pending
76,100-1514-Delhi,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-24 11:00:00,2024-07-24 10:32:27,2024-07-24 10:52:02,13.0,45,On-time pickup
75,100-1514-Delhi,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-23 17:30:00,2024-07-23 21:23:48,2024-07-23 21:40:36,69.0,13,Late pickup
74,100-1514-Delhi,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-23 11:00:00,2024-07-23 10:56:09,2024-07-23 11:31:09,64.0,61,On-time pickup
73,100-1514-Delhi,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-22 17:30:00,2024-07-22 17:34:17,2024-07-22 17:38:34,23.0,45,On-time pickup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-03 16:00:00,2024-07-03 17:27:46,2024-07-03 17:27:46,1.0,0,Late pickup
6009,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-02 16:00:00,2024-07-02 18:00:52,2024-07-02 18:00:52,1.0,2,Late pickup
6008,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-01 16:00:00,2024-07-01 17:19:16,2024-07-01 17:19:34,7.0,7,Late pickup
6007,87-327-Mumbai,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-06-30 16:00:00,NaT,NaT,NaN,0,Missed


In [ ]:
final['key'] = final.apply(lambda row: f"{row['key']}-{row['pickup_cutoff'].strftime('%d/%m/%y/%H')}", axis=1)


In [ ]:
print(final[final['key'] == '100-329-Bangalore-17/07/24/18'])
print(final[final['key'] == '332-1540-Hyderabad-18/07/24/20'])
display(final)

                              key  warehouse_id  \
96  100-329-Bangalore-17/07/24/18           329   

                         warehouse_name user_name  user_id warehouse_city  \
96  Indiejewel Fashions Private Limited      GIVA      100      Bangalore   

   accounts_poc_number fm_poc_number client_poc_number ach_number  \
96                        9066859089        8548848808              

         pickup_cutoff   pickup_start_time     pickup_end_time  actual_load  \
96 2024-07-17 18:30:00 2024-07-17 21:19:14 2024-07-18 10:58:59        170.0   

    expected_load pickup_status  
96            178   Late pickup  
                                 key  warehouse_id warehouse_name  user_name  \
1501  332-1540-Hyderabad-18/07/24/20          1540  Decathlon HYD  Decathlon   

      user_id warehouse_city accounts_poc_number fm_poc_number  \
1501      332      Hyderabad                        8008886275   

     client_poc_number ach_number       pickup_cutoff   pickup_start_time  \
1501 

,key,warehouse_id,warehouse_name,user_name,user_id,warehouse_city,accounts_poc_number,fm_poc_number,client_poc_number,ach_number,pickup_cutoff,pickup_start_time,pickup_end_time,actual_load,expected_load,pickup_status
77,100-1514-Delhi-24/07/24/17,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-24 17:30:00,NaT,NaT,NaN,0,Pending
76,100-1514-Delhi-24/07/24/11,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-24 11:00:00,2024-07-24 10:32:27,2024-07-24 10:52:02,13.0,45,On-time pickup
75,100-1514-Delhi-23/07/24/17,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-23 17:30:00,2024-07-23 21:23:48,2024-07-23 21:40:36,69.0,13,Late pickup
74,100-1514-Delhi-23/07/24/11,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-23 11:00:00,2024-07-23 10:56:09,2024-07-23 11:31:09,64.0,61,On-time pickup
73,100-1514-Delhi-22/07/24/17,1514,GIVA DEL,GIVA,100,Delhi,,8287823040,7503668054,,2024-07-22 17:30:00,2024-07-22 17:34:17,2024-07-22 17:38:34,23.0,45,On-time pickup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,87-327-Mumbai-03/07/24/16,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-03 16:00:00,2024-07-03 17:27:46,2024-07-03 17:27:46,1.0,0,Late pickup
6009,87-327-Mumbai-02/07/24/16,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-02 16:00:00,2024-07-02 18:00:52,2024-07-02 18:00:52,1.0,2,Late pickup
6008,87-327-Mumbai-01/07/24/16,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-07-01 16:00:00,2024-07-01 17:19:16,2024-07-01 17:19:34,7.0,7,Late pickup
6007,87-327-Mumbai-30/06/24/16,327,GOAT,The Label Life,87,Mumbai,,,9996237317,,2024-06-30 16:00:00,NaT,NaT,NaN,0,Missed


In [ ]:
filepath = 'final_fm.xlsx'
final.to_excel(filepath, index=False)

In [ ]:
# def get_conn_string(SSH_required, key_path):
#     db = 'datawarehouse'
#     DB_HOST = 'datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
#     conn_string = None

#     if SSH_required == 'Yes':
#         SSH_HOST = 'ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
#         ssh_tunnel = SSHTunnelForwarder(
#             (SSH_HOST, 22),
#             ssh_username="ec2-user",
#             ssh_private_key=key_path,
#             ssh_private_key_password="",
#             remote_bind_address=(DB_HOST, 5432),
#             local_bind_address=('127.0.0.1', 0)
#         )
#         print('Tunnel Started')
#         ssh_tunnel.start()
#         conn_string = f"postgresql://postgres:Simply1234@{ssh_tunnel.local_bind_host}:{ssh_tunnel.local_bind_port}/postgres"
#     else:
#         conn_string = f"postgresql://postgres:Simply1234@{DB_HOST}:5432/postgres"
    
#     print('Connection String Created')
#     return conn_string

# # Set your parameters for SSH and key path
# SSH_required = 'Yes'
# key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'

# # Get the connection string
# connection_string = get_conn_string(SSH_required, key_path)

# # Create the SQLAlchemy engine
# engine = create_engine(connection_string)

Tunnel Started
Connection String Created


In [ ]:
# # Taking the dataframe to sql table in the database using sqlalchemy and pandas:

# # Used this commented code to create the table in public schema:
# metadata = MetaData(schema='public')

# fm_status = Table(
#     'fm_status', metadata,
#     Column('key', String, primary_key=True),
#     Column('warehouse_id', String),
#     Column('warehouse_name', String),
#     Column('user_name', String),
#     Column('user_id', Integer),
#     Column('warehouse_city', String),
#     Column('accounts_poc_number', String),  # Added new columns
#     Column('fm_poc_number', String),
#     Column('client_poc_number', String),
#     Column('ach_number', String),
#     Column('pickup_cutoff', DateTime),
#     Column('pickup_start_time', DateTime),
#     Column('pickup_end_time', DateTime),
#     Column('actual_load', Float),
#     Column('expected_load', Float),
#     Column('pickup_status', String)
# )


# # Create the table in the database
# metadata.create_all(engine)

# # Insert the DataFrame into the SQL table, specifying the schema
# final.to_sql('fm_status', con=engine, index=False, if_exists='replace', schema='public')

# # Close the connection
# engine.dispose()

2024-07-24 14:09:45,643| ERROR   | Socket exception: No route to host (65)
